# Introduction

Since Jan. 1, 2015, [The Washington Post](https://www.washingtonpost.com/) has been compiling a database of every fatal shooting in the US by a police officer in the line of duty. 

<center><img src=https://i.imgur.com/sX3K62b.png></center>

While there are many challenges regarding data collection and reporting, The Washington Post has been tracking more than a dozen details about each killing. This includes the race, age and gender of the deceased, whether the person was armed, and whether the victim was experiencing a mental-health crisis. The Washington Post has gathered this supplemental information from law enforcement websites, local new reports, social media, and by monitoring independent databases such as "Killed by police" and "Fatal Encounters". The Post has also conducted additional reporting in many cases.

There are 4 additional datasets: US census data on poverty rate, high school graduation rate, median household income, and racial demographics. [Source of census data](https://factfinder.census.gov/faces/nav/jsf/pages/community_facts.xhtml).

### Upgrade Plotly

Run the cell below if you are working with Google Colab

In [1]:
# %pip install --upgrade plotly

## Import Statements

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import re

# This might be helpful:
from collections import Counter

## Notebook Presentation

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

## Load the Data

In [4]:
df_hh_income = pd.read_csv('Median_Household_Income_2015.csv', encoding="windows-1252")
df_pct_poverty = pd.read_csv('Pct_People_Below_Poverty_Level.csv', encoding="windows-1252")
df_pct_completed_hs = pd.read_csv('Pct_Over_25_Completed_High_School.csv', encoding="windows-1252")
df_share_race_city = pd.read_csv('Share_of_Race_By_City.csv', encoding="windows-1252")
df_fatalities = pd.read_csv('Deaths_by_Police_US.csv', encoding="windows-1252")

# Preliminary Data Exploration

* What is the shape of the DataFrames? 
* How many rows and columns do they have?
* What are the column names?
* Are there any NaN values or duplicates?

In [5]:
df_hh_income

,Geographic Area,City,Median Income
0,AL,Abanda CDP,11207
1,AL,Abbeville city,25615
2,AL,Adamsville city,42575
3,AL,Addison town,37083
4,AL,Akron town,21667
...,...,...,...
29317,WY,Woods Landing-Jelm CDP,NaN
29318,WY,Worland city,41523
29319,WY,Wright town,77114
29320,WY,Yoder town,37500


In [6]:
df_hh_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29322 entries, 0 to 29321
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Geographic Area  29322 non-null  object
 1   City             29322 non-null  object
 2   Median Income    29271 non-null  object
dtypes: object(3)
memory usage: 687.4+ KB


In [7]:
df_hh_income.duplicated().values.any()

False

In [8]:
df_pct_poverty

,Geographic Area,City,poverty_rate
0,AL,Abanda CDP,78.8
1,AL,Abbeville city,29.1
2,AL,Adamsville city,25.5
3,AL,Addison town,30.7
4,AL,Akron town,42
...,...,...,...
29324,WY,Woods Landing-Jelm CDP,18.6
29325,WY,Worland city,15.3
29326,WY,Wright town,5.9
29327,WY,Yoder town,5.4


In [9]:
df_pct_poverty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29329 entries, 0 to 29328
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Geographic Area  29329 non-null  object
 1   City             29329 non-null  object
 2   poverty_rate     29329 non-null  object
dtypes: object(3)
memory usage: 687.5+ KB


In [10]:
df_pct_poverty.duplicated().values.any()

False

In [11]:
df_pct_completed_hs

,Geographic Area,City,percent_completed_hs
0,AL,Abanda CDP,21.2
1,AL,Abbeville city,69.1
2,AL,Adamsville city,78.9
3,AL,Addison town,81.4
4,AL,Akron town,68.6
...,...,...,...
29324,WY,Woods Landing-Jelm CDP,100
29325,WY,Worland city,85.6
29326,WY,Wright town,89.2
29327,WY,Yoder town,79.4


In [12]:
df_pct_completed_hs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29329 entries, 0 to 29328
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Geographic Area       29329 non-null  object
 1   City                  29329 non-null  object
 2   percent_completed_hs  29329 non-null  object
dtypes: object(3)
memory usage: 687.5+ KB


In [13]:
df_pct_completed_hs.duplicated().values.any()

False

In [14]:
df_share_race_city

,Geographic area,City,share_white,share_black,share_native_american,share_asian,share_hispanic
0,AL,Abanda CDP,67.2,30.2,0,0,1.6
1,AL,Abbeville city,54.4,41.4,0.1,1,3.1
2,AL,Adamsville city,52.3,44.9,0.5,0.3,2.3
3,AL,Addison town,99.1,0.1,0,0.1,0.4
4,AL,Akron town,13.2,86.5,0,0,0.3
...,...,...,...,...,...,...,...
29263,WY,Woods Landing-Jelm CDP,95.9,0,0,2.1,0
29264,WY,Worland city,89.9,0.3,1.3,0.6,16.6
29265,WY,Wright town,94.5,0.1,1.4,0.2,6.2
29266,WY,Yoder town,97.4,0,0,0,4


In [15]:
df_share_race_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29268 entries, 0 to 29267
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Geographic area        29268 non-null  object
 1   City                   29268 non-null  object
 2   share_white            29268 non-null  object
 3   share_black            29268 non-null  object
 4   share_native_american  29268 non-null  object
 5   share_asian            29268 non-null  object
 6   share_hispanic         29268 non-null  object
dtypes: object(7)
memory usage: 1.6+ MB


In [16]:
df_share_race_city.duplicated().values.any()

False

In [17]:
df_fatalities

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,02/01/15,shot,gun,53.00,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,02/01/15,shot,gun,47.00,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,03/01/15,shot and Tasered,unarmed,23.00,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,04/01/15,shot,toy weapon,32.00,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,04/01/15,shot,nail gun,39.00,M,H,Evans,CO,False,attack,Not fleeing,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530,2822,Rodney E. Jacobs,28/07/17,shot,gun,31.00,M,NaN,Kansas City,MO,False,attack,Not fleeing,False
2531,2813,TK TK,28/07/17,shot,vehicle,NaN,M,NaN,Albuquerque,NM,False,attack,Car,False
2532,2818,Dennis W. Robinson,29/07/17,shot,gun,48.00,M,NaN,Melba,ID,False,attack,Car,False
2533,2817,Isaiah Tucker,31/07/17,shot,vehicle,28.00,M,B,Oshkosh,WI,False,attack,Car,True


In [18]:
df_fatalities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2535 entries, 0 to 2534
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       2535 non-null   int64  
 1   name                     2535 non-null   object 
 2   date                     2535 non-null   object 
 3   manner_of_death          2535 non-null   object 
 4   armed                    2526 non-null   object 
 5   age                      2458 non-null   float64
 6   gender                   2535 non-null   object 
 7   race                     2340 non-null   object 
 8   city                     2535 non-null   object 
 9   state                    2535 non-null   object 
 10  signs_of_mental_illness  2535 non-null   bool   
 11  threat_level             2535 non-null   object 
 12  flee                     2470 non-null   object 
 13  body_camera              2535 non-null   bool   
dtypes: bool(2), float64(1), 

In [19]:
df_fatalities.duplicated().values.any()

False

## datatypes: df_hh_income

In [20]:
df_hh_income

,Geographic Area,City,Median Income
0,AL,Abanda CDP,11207
1,AL,Abbeville city,25615
2,AL,Adamsville city,42575
3,AL,Addison town,37083
4,AL,Akron town,21667
...,...,...,...
29317,WY,Woods Landing-Jelm CDP,NaN
29318,WY,Worland city,41523
29319,WY,Wright town,77114
29320,WY,Yoder town,37500


In [21]:
df_hh_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29322 entries, 0 to 29321
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Geographic Area  29322 non-null  object
 1   City             29322 non-null  object
 2   Median Income    29271 non-null  object
dtypes: object(3)
memory usage: 687.4+ KB


In [22]:
filt = df_hh_income['Median Income'] == '(X)'
df_hh_income[filt] 

,Geographic Area,City,Median Income
44,AL,Benton town,(X)
56,AL,Bon Secour CDP,(X)
57,AL,Boykin CDP,(X)
65,AL,Bristow Cove CDP,(X)
78,AL,Cardiff town,(X)
...,...,...,...
28476,WI,Clam Lake CDP,(X)
28608,WI,Glen Flora village,(X)
28960,WI,Saxon CDP,(X)
28992,WI,Springfield CDP,(X)


In [23]:
df_hh_income.loc[filt, 'Median Income'] = np.nan

In [24]:
filt = df_hh_income['Median Income'] == '-'
df_hh_income[filt] 

,Geographic Area,City,Median Income
71,AL,Bucks CDP,-
74,AL,Calvert CDP,-
84,AL,Catherine CDP,-
338,AL,Megargel CDP,-
462,AL,Rockville CDP,-
...,...,...,...
28258,WV,Roderfield CDP,-
28299,WV,Tioga CDP,-
28311,WV,Vivian CDP,-
28332,WV,Whitmer CDP,-


In [25]:
df_hh_income.loc[filt, 'Median Income'] = np.nan

In [26]:
df_hh_income['Median Income'] = df_hh_income['Median Income'].str.replace(',','')
df_hh_income['Median Income'] = df_hh_income['Median Income'].str.replace('-','')
df_hh_income['Median Income'] = df_hh_income['Median Income'].str.replace('+','')

In [27]:
df_hh_income['Median Income'] = pd.to_numeric(df_hh_income['Median Income'])

In [28]:
df_hh_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29322 entries, 0 to 29321
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Geographic Area  29322 non-null  object 
 1   City             29322 non-null  object 
 2   Median Income    27418 non-null  float64
dtypes: float64(1), object(2)
memory usage: 687.4+ KB


## Datatypes: df_pct_poverty

In [29]:
df_pct_poverty

,Geographic Area,City,poverty_rate
0,AL,Abanda CDP,78.8
1,AL,Abbeville city,29.1
2,AL,Adamsville city,25.5
3,AL,Addison town,30.7
4,AL,Akron town,42
...,...,...,...
29324,WY,Woods Landing-Jelm CDP,18.6
29325,WY,Worland city,15.3
29326,WY,Wright town,5.9
29327,WY,Yoder town,5.4


In [30]:
filt = df_pct_poverty['poverty_rate'] == '-'
df_pct_poverty[filt]

,Geographic Area,City,poverty_rate
573,AL,Whatley CDP,-
608,AK,Attu Station CDP,-
632,AK,Chicken CDP,-
637,AK,Chisana CDP,-
662,AK,Dot Lake CDP,-
...,...,...,...
29261,WY,Oakley CDP,-
29266,WY,Owl Creek CDP,-
29273,WY,Powder River CDP,-
29289,WY,Ryan Park CDP,-


In [31]:
df_pct_poverty.loc[filt, 'poverty_rate']= np.nan
# df.loc[df['country'] == 'Yellow Sea', 'country'] = 'China'

In [32]:
df_pct_poverty.poverty_rate = pd.to_numeric(df_pct_poverty.poverty_rate)

In [33]:
df_pct_poverty.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29329 entries, 0 to 29328
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Geographic Area  29329 non-null  object 
 1   City             29329 non-null  object 
 2   poverty_rate     29128 non-null  float64
dtypes: float64(1), object(2)
memory usage: 687.5+ KB


## Datatypes: df_pct_completed_hs

In [34]:
df_pct_completed_hs

,Geographic Area,City,percent_completed_hs
0,AL,Abanda CDP,21.2
1,AL,Abbeville city,69.1
2,AL,Adamsville city,78.9
3,AL,Addison town,81.4
4,AL,Akron town,68.6
...,...,...,...
29324,WY,Woods Landing-Jelm CDP,100
29325,WY,Worland city,85.6
29326,WY,Wright town,89.2
29327,WY,Yoder town,79.4


In [35]:
df_pct_completed_hs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29329 entries, 0 to 29328
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Geographic Area       29329 non-null  object
 1   City                  29329 non-null  object
 2   percent_completed_hs  29329 non-null  object
dtypes: object(3)
memory usage: 687.5+ KB


In [36]:
df_pct_completed_hs['percent_completed_hs'] = df_pct_completed_hs['percent_completed_hs'].replace('[-+$,a-z]', '', regex=True)

In [37]:
df_pct_completed_hs['percent_completed_hs'] = pd.to_numeric(df_pct_completed_hs['percent_completed_hs'])

In [38]:
df_pct_completed_hs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29329 entries, 0 to 29328
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Geographic Area       29329 non-null  object 
 1   City                  29329 non-null  object 
 2   percent_completed_hs  29132 non-null  float64
dtypes: float64(1), object(2)
memory usage: 687.5+ KB


## Datatype: df_share_race_city

In [39]:
df_share_race_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29268 entries, 0 to 29267
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Geographic area        29268 non-null  object
 1   City                   29268 non-null  object
 2   share_white            29268 non-null  object
 3   share_black            29268 non-null  object
 4   share_native_american  29268 non-null  object
 5   share_asian            29268 non-null  object
 6   share_hispanic         29268 non-null  object
dtypes: object(7)
memory usage: 1.6+ MB


In [40]:
df_share_race_city

,Geographic area,City,share_white,share_black,share_native_american,share_asian,share_hispanic
0,AL,Abanda CDP,67.2,30.2,0,0,1.6
1,AL,Abbeville city,54.4,41.4,0.1,1,3.1
2,AL,Adamsville city,52.3,44.9,0.5,0.3,2.3
3,AL,Addison town,99.1,0.1,0,0.1,0.4
4,AL,Akron town,13.2,86.5,0,0,0.3
...,...,...,...,...,...,...,...
29263,WY,Woods Landing-Jelm CDP,95.9,0,0,2.1,0
29264,WY,Worland city,89.9,0.3,1.3,0.6,16.6
29265,WY,Wright town,94.5,0.1,1.4,0.2,6.2
29266,WY,Yoder town,97.4,0,0,0,4


In [41]:
df_share_race_city['share_white'] = df_share_race_city['share_white'].replace('[-+$,a-zA-Z \(\)]', '', regex=True) 
df_share_race_city['share_black'] = df_share_race_city['share_black'].replace('[-+$,a-zA-Z \(\)]', '', regex=True) 
df_share_race_city['share_native_american'] = df_share_race_city['share_native_american'].replace('[-+$,a-zA-Z \(\)]', '', regex=True) 
df_share_race_city['share_asian'] = df_share_race_city['share_asian'].replace('[-+$,a-zA-Z \(\)]', '', regex=True) 
df_share_race_city['share_hispanic'] = df_share_race_city['share_hispanic'].replace('[-+$,a-zA-Z \(\)]', '', regex=True) 

In [42]:
df_share_race_city['share_white'] = pd.to_numeric(df_share_race_city['share_white'])
df_share_race_city['share_black'] = pd.to_numeric(df_share_race_city['share_black'])
df_share_race_city['share_native_american'] = pd.to_numeric(df_share_race_city['share_native_american'])
df_share_race_city['share_asian'] = pd.to_numeric(df_share_race_city['share_asian'])
df_share_race_city['share_hispanic'] = pd.to_numeric(df_share_race_city['share_hispanic'])


In [43]:
df_share_race_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29268 entries, 0 to 29267
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Geographic area        29268 non-null  object 
 1   City                   29268 non-null  object 
 2   share_white            29248 non-null  float64
 3   share_black            29248 non-null  float64
 4   share_native_american  29248 non-null  float64
 5   share_asian            29248 non-null  float64
 6   share_hispanic         29248 non-null  float64
dtypes: float64(5), object(2)
memory usage: 1.6+ MB


## Datatypes: df_fatalities

In [44]:
df_fatalities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2535 entries, 0 to 2534
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       2535 non-null   int64  
 1   name                     2535 non-null   object 
 2   date                     2535 non-null   object 
 3   manner_of_death          2535 non-null   object 
 4   armed                    2526 non-null   object 
 5   age                      2458 non-null   float64
 6   gender                   2535 non-null   object 
 7   race                     2340 non-null   object 
 8   city                     2535 non-null   object 
 9   state                    2535 non-null   object 
 10  signs_of_mental_illness  2535 non-null   bool   
 11  threat_level             2535 non-null   object 
 12  flee                     2470 non-null   object 
 13  body_camera              2535 non-null   bool   
dtypes: bool(2), float64(1), 

In [45]:
df_fatalities['date'] = pd.to_datetime(df_fatalities['date'], format='%d/%m/%y') 

In [46]:
df_fatalities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2535 entries, 0 to 2534
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       2535 non-null   int64         
 1   name                     2535 non-null   object        
 2   date                     2535 non-null   datetime64[ns]
 3   manner_of_death          2535 non-null   object        
 4   armed                    2526 non-null   object        
 5   age                      2458 non-null   float64       
 6   gender                   2535 non-null   object        
 7   race                     2340 non-null   object        
 8   city                     2535 non-null   object        
 9   state                    2535 non-null   object        
 10  signs_of_mental_illness  2535 non-null   bool          
 11  threat_level             2535 non-null   object        
 12  flee                     2470 non-

In [47]:
df_fatalities

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,2015-01-02,shot,gun,53.00,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.00,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.00,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.00,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.00,M,H,Evans,CO,False,attack,Not fleeing,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530,2822,Rodney E. Jacobs,2017-07-28,shot,gun,31.00,M,NaN,Kansas City,MO,False,attack,Not fleeing,False
2531,2813,TK TK,2017-07-28,shot,vehicle,NaN,M,NaN,Albuquerque,NM,False,attack,Car,False
2532,2818,Dennis W. Robinson,2017-07-29,shot,gun,48.00,M,NaN,Melba,ID,False,attack,Car,False
2533,2817,Isaiah Tucker,2017-07-31,shot,vehicle,28.00,M,B,Oshkosh,WI,False,attack,Car,True


## Data Cleaning - Check for Missing Values and Duplicates

Consider how to deal with the NaN values. Perhaps substituting 0 is appropriate. 

# Chart the Poverty Rate in each US State

Create a bar chart that ranks the poverty rate from highest to lowest by US state. Which state has the highest poverty rate? Which state has the lowest poverty rate?  Bar Plot

In [48]:
df_pct_poverty.groupby('Ge')

KeyError: 'Ge'

In [ ]:
df_pct_poverty = df_pct_poverty.rename(columns={"Geographic Area":"State"})

In [ ]:
avg_poverty_per_state = df_pct_poverty.groupby('State').agg(avg_poverty=('poverty_rate', 'mean')).sort_values('avg_poverty', ascending=False)
# .agg(launches=('year', 'count'))
avg_poverty_per_state

In [ ]:
plt.bar(avg_poverty_per_state.index, avg_poverty_per_state.avg_poverty)

# Chart the High School Graduation Rate by US State

Show the High School Graduation Rate in ascending order of US States. Which state has the lowest high school graduation rate? Which state has the highest?

In [ ]:
df_pct_completed_hs = df_pct_completed_hs.rename(columns={"Geographic Area":"State"})
df_pct_completed_hs

In [ ]:
avg_hs_per_state = df_pct_completed_hs.groupby('State').agg(avg_hs_per_state=('percent_completed_hs', 'mean')).sort_values('avg_hs_per_state', ascending=False)
avg_hs_per_state

In [ ]:
plt.bar(avg_hs_per_state.index, avg_hs_per_state.avg_hs_per_state)

# Visualise the Relationship between Poverty Rates and High School Graduation Rates

#### Create a line chart with two y-axes to show if the rations of poverty and high school graduation move together.  

In [ ]:
fig, ax1 = plt.subplots()

color="tab:red"
ax1.set_xlabel('state')
ax1.set_ylabel('poverty', color=color)
ax1.plot(avg_poverty_per_state.index , avg_poverty_per_state.avg_poverty , color=color)
ax1.plot(avg_poverty_per_state.avg_poverty, label="poverty rate", color=color)

ax2 = ax1.twinx()

color="tab:blue"
ax2.set_ylabel('highschool success', color=color,  rotation=270,labelpad=10)
ax2.plot(avg_hs_per_state.index, avg_hs_per_state.avg_hs_per_state, color=color)
ax2.plot(avg_hs_per_state.avg_hs_per_state, label="high school", color=color)

# this would place the legend OUTSIDE the plot
# fig.legend(loc="upper left")

# from https://stackoverflow.com/questions/4700614/how-to-put-the-legend-outside-the-plot
# the exact numbers were trial and error
fig.legend(bbox_to_anchor=(0.41, 0.87))


#### Now use a Seaborn .jointplot() with a Kernel Density Estimate (KDE) and/or scatter plot to visualise the same relationship

In [ ]:
concat = pd.merge(avg_poverty_per_state, avg_hs_per_state, on='State')
concat


In [ ]:
sns.scatterplot(data=concat.avg_poverty, color="g")
ax2 = plt.twinx()
sns.scatterplot(data=concat.avg_hs_per_state, color="b", ax=ax2)

#### Seaborn's `.lmplot()` or `.regplot()` to show a linear regression between the poverty ratio and the high school graduation ratio. 

In [ ]:
# sns.regplot(data=concat.avg_poverty, color="g")
# ax2 = plt.twinx()
# sns.regplot(data=concat.avg_hs_per_state, color="b", ax=ax2)

sns.regplot(data=concat, 
            x='avg_poverty',
            y='avg_hs_per_state')

# Create a Bar Chart with Subsections Showing the Racial Makeup of Each US State

Visualise the share of the white, black, hispanic, asian and native american population in each US State using a bar chart with sub sections. 

In [ ]:
df_share_race_city = df_share_race_city.rename(columns={"Geographic area":"State"}) 

In [ ]:
df_share_race_city

In [ ]:
df_race = df_share_race_city.drop(['City'], axis=1)
avg_race_state = df_race.groupby('State').mean()
avg_race_state

In [ ]:
import plotly.graph_objects as go
# Create dummy data indexed by month and with multi-columns [product, revenue]
index = ["California", "Texas", "Arizona", "Nevada", "Louisiana"]
df = pd.concat(
    [
        pd.DataFrame(
            np.random.rand(5, 3) * 1.25 + 0.25,
            index=index,
            columns=["Revenue1", "Revenue2", "Revenue3"]
        ),
#         pd.DataFrame(
#             np.random.rand(5, 3) + 0.5,
#             index=index,
#             columns=["Revenue1", "Revenue2", "Revenue3"]
#         ),
    ],
    axis=1,
    keys=["Product1"] #, "Product2"]
)

# Create a figure with the right layout
fig = go.Figure(
    layout=go.Layout(
        height=600,
        width=1000,
        barmode="relative",
        yaxis_showticklabels=False,
        yaxis_showgrid=False,
        yaxis_range=[0, df.groupby(axis=1, level=0).sum().max().max() * 1.5],
       # Secondary y-axis overlayed on the primary one and not visible
#         yaxis2=go.layout.YAxis(
#             visible=False,
#             matches="y",
#             overlaying="y",
#             anchor="x",
#         ),
        font=dict(size=24),
        legend_x=0,
        legend_y=1,
        legend_orientation="h",
        hovermode="x",
        margin=dict(b=0,t=10,l=0,r=10)
    )
)

# Define some colors for the product, revenue pairs
colors = {
    "Product1": {
        "Revenue1": "#F28F1D",
        "Revenue2": "#F6C619",
        "Revenue3": "#FADD75",
    } # ,
#     "Product2": {
#         "Revenue1": "#2B6045",
#         "Revenue2": "#5EB88A",
#         "Revenue3": "#9ED4B9",
#     }
}

# Add the traces
for i, t in enumerate(colors):
    for j, col in enumerate(df[t].columns):
        if (df[t][col] == 0).all():
            continue
        fig.add_bar(
            x=df.index,
            y=df[t][col],
            # Set the right yaxis depending on the selected product (from enumerate)
            yaxis=f"y{i + 1}",
            # Offset the bar trace, offset needs to match the width
            # For categorical traces, each category is spaced by 1
            offsetgroup=str(i),
            offset=(i - 1) * 1/3,
            width=1/3,
            legendgroup=t,
            legendgrouptitle_text=t,
            name=col,
            marker_color=colors[t][col],
            marker_line=dict(width=2, color="#333"),
            hovertemplate="%{y}<extra></extra>"
        )

fig.show()

In [ ]:
avg_race_state = avg_race_state.reset_index()

In [ ]:
df_race_long = pd.melt(avg_race_state, id_vars='State', value_vars=['share_white', 'share_black', 'share_native_american', 'share_asian', 'share_hispanic'],
             var_name='race', value_name='pct_share')

In [ ]:
avg_race_state.columns

In [ ]:
df_race_long

In [ ]:
fig = px.bar(df_race_long, x = 'State', y = 'pct_share', color = 
    'race', barmode = 'stack')
fig.update_layout(title = "Education Level Customers' Composition",
     xaxis_title = 'Customer Condition', yaxis_title = 'Counts', 
     width = 1600, height = 1000)
fig.show()

In [ ]:
df_share_race_city.loc[df_share_race_city.State == 'TX']

The data are crap, many percentages do not add up to 100, eg: 
- Zarate CDP, TX, is both 100% white and 100% hispanic
- Abram CDP, TX, is 99% white and 99% hispanic

# Create Donut Chart by of People Killed by Race

Hint: Use `.value_counts()`

In [ ]:
df_fatalities.gender.replace(['M','F'],['Male','Female'],inplace=True)
df_fatalities.race.replace(['W','B', 'N', 'A', 'H', 'O'],['White','Black', 'Native_American', 'Asian', 'Hispanic'],inplace=True)
df_fatalities.race.fillna("Not_recorded", inplace=True)

In [ ]:
df_fatalities.race.nunique().values_count()

In [ ]:
df_fatalities_per_race = df_fatalities.groupby('race').agg(killings=('race', 'count'))
df_fatalities_per_race

In [ ]:
fig = px.pie(labels=df_fatalities_per_race.index,
values=df_fatalities_per_race.killings,
title="killings per race",
names=df_fatalities_per_race.index,
             hole=0.8
)
fig.update_traces(textposition='outside', textinfo='percent+label')
 
fig.show()

# Create a Chart Comparing the Total Number of Deaths of Men and Women

Use `df_fatalities` to illustrate how many more men are killed compared to women. 

In [ ]:
df_fatalities_per_gender = df_fatalities.groupby('gender').agg(killings=('gender', 'count'))
df_fatalities_per_gender

In [ ]:
fig = px.pie(labels=df_fatalities_per_gender.index,
values=df_fatalities_per_gender.killings,
title="killings per gender",
names=df_fatalities_per_gender.index,
)
fig.update_traces(textposition='outside', textinfo='percent+label')
 
fig.show()

# Create a Box Plot Showing the Age and Manner of Death

Break out the data by gender using `df_fatalities`. Is there a difference between men and women in the manner of death? 

In [ ]:
df_fatalities_per_gender_method = df_fatalities[['id','manner_of_death', 'gender']]
df_fatalities_per_gender_method

In [ ]:
df_fatalities_per_gender_method.groupby(['manner_of_death','gender']).count()

In [ ]:
10200/(102+2261)

In [ ]:
500/(167+5)

4.3% of the people shot are women, 2.9% of people tasered and shot are women

In [ ]:
box = px.box(df_fatalities_per_gender_method,
             y='id',
             x='gender',
             color='manner_of_death',
             notched=True,
             points='all',
             title='gender differences in manner of killing')
 
box.update_layout(yaxis=dict(type='log'))
 
box.show()

# Were People Armed? 

In what percentage of police killings were people armed? Create chart that show what kind of weapon (if any) the deceased was carrying. How many of the people killed by police were armed with guns versus unarmed? 

In [ ]:
df_fatalities

In [ ]:
filt = df_armed['weapon_count'] < 10
other = df_armed[filt].agg(other_weapons=('weapon_count', 'sum'))
other

In [ ]:
df_armed = df_fatalities.groupby('armed').agg(weapon_count=('id', 'count')).sort_values('weapon_count', ascending=False).reset_index()
df_armed_10 = df_armed[:8]
df_armed_10 = df_armed_10._append({'armed':'other', 'weapon_count':150},ignore_index=True)
df_armed_other = df_armed[9:]

In [ ]:
df_armed_10

In [ ]:
fig = px.pie(labels=df_armed_10.index,
values=df_armed_10.weapon_count,
title="armed victims",
names=df_armed_10.armed,
)
fig.update_traces(textposition='outside', textinfo='percent+label')
 
fig.show()

In [ ]:
fig = px.pie(labels=df_armed.index[9:],
values=df_armed.weapon_count[9:],
title="armed victims",
names=df_armed.index[9:],
)
fig.update_traces(textposition='outside', textinfo='percent+label')
 
fig.show()

# How Old Were the People Killed?

Work out what percentage of people killed were under 25 years old.  

In [ ]:
filt = df_fatalities.age > 0
df_ages = df_fatalities[filt]
df_ages

In [ ]:
filt = df_ages['age'] < 25
df_age_under_25 = df_ages[filt]
df_age_under_25

In [ ]:
print(f"under 25 = {4500/2458}%")

Create a histogram and KDE plot that shows the distribution of ages of the people killed by police. 

In [ ]:
sns.displot(df_ages['age'], 
            bins=50, 
            aspect=2,
            kde=True, 
            color='#2196f3',
           log_scale=True)

# plt.title(f'1970s Home Values in Boston. Average: ${(1000*data.PRICE.mean()):.6}')
plt.xlabel('age of victims')
plt.ylabel('Nr. of victims')

plt.show()

In [ ]:
plt.figure(dpi=200)
sns.kdeplot(df_ages.age, fill=True)
# sns.kdeplot(after_washing.pct_deaths, shade=True)
plt.title('ages')
plt.show()

Create a seperate KDE plot for each race. Is there a difference between the distributions? 

In [ ]:
filt = df_ages['race'] == "White"
plt.figure(dpi=200)
sns.kdeplot(df_ages[filt].age, fill=True)
# sns.kdeplot(after_washing.pct_deaths, shade=True)
plt.title('ages')
plt.show()

In [ ]:
filt = df_ages['race'] == "Black"
plt.figure(dpi=200)
sns.kdeplot(df_ages[filt].age, fill=True)
# sns.kdeplot(after_washing.pct_deaths, shade=True)
plt.title('ages')
plt.show()

In [ ]:
filt = df_ages['race'] == "Native_American"
plt.figure(dpi=200)
sns.kdeplot(df_ages[filt].age, fill=True)
# sns.kdeplot(after_washing.pct_deaths, shade=True)
plt.title('ages')
plt.show()

In [ ]:
filt = df_ages['race'] == "Asian"
plt.figure(dpi=200)
sns.kdeplot(df_ages[filt].age, fill=True)
# sns.kdeplot(after_washing.pct_deaths, shade=True)
plt.title('ages')
plt.show()

In [ ]:
filt = df_ages['race'] == "Hispanic"
plt.figure(dpi=200)
sns.kdeplot(df_ages[filt].age, fill=True)
# sns.kdeplot(after_washing.pct_deaths, shade=True)
plt.title('ages')
plt.show()

# Race of People Killed

Create a chart that shows the total number of people killed by race. 

In [ ]:
df_killed_by_race = df_fatalities.groupby('race').agg(killed=('race', 'count'))
df_killed_by_race

In [ ]:
fig = px.pie(labels=df_killed_by_race.index,
values=df_killed_by_race.killed,
title="killed by race",
names=df_killed_by_race.index,
)
fig.update_traces(textposition='outside', textinfo='percent+label')
 
fig.show()

# Mental Illness and Police Killings

What percentage of people killed by police have been diagnosed with a mental illness?

In [ ]:
#df['sales'] / df.groupby('state')['sales'].transform('sum')
df_mental = df_fatalities.groupby('signs_of_mental_illness').agg(ill_count=('signs_of_mental_illness','count'))
df_mental

In [ ]:
fig = px.pie(labels=df_mental.index,
values=df_mental.ill_count,
title="df_mental",
names=df_mental.index,
)
fig.update_traces(textposition='outside', textinfo='percent+label')
 
fig.show()

# In Which Cities Do the Most Police Killings Take Place?

Create a chart ranking the top 10 cities with the most police killings. Which cities are the most dangerous?  

In [ ]:
df_cities = df_fatalities.groupby('city').agg(killings=('city', 'count')).sort_values('killings', ascending=False)[:10]
df_cities

In [ ]:
plt.bar(df_cities.index, df_cities.killings)

# Rate of Death by Race

Find the share of each race in the top 10 cities. Contrast this with the top 10 cities of police killings to work out the rate at which people are killed by race for each city. 

In [ ]:
# df_fatalities.gender.replace(['M','F'],['Male','Female'],inplace=True)
# df_fatalities.race.replace(['W','B', 'N', 'A', 'H'],['White','Black', 'Native_American', 'Asian', 'Hispanic'],inplace=True)
# df_fatalities.race.fillna("Not_recorded", inplace=True

In [ ]:

                         )


In [ ]:
cities = ['Los Angeles', 'Phoenix', 'Houston', 'Chicago', 'Las Vegas', 'San Antonio', 'Columbus', 'Miami', 'Austin', 'St. Louis']
df_top10_cities = df_fatalities[df_fatalities['city'].isin(cities)]
df_top10_cities_per_race = df_top10_cities.groupby(['city', 'race']).agg(killings=('race', 'count'))
df_top10_cities_per_race
df_top10_cities_per_race = df_top10_cities_per_race.reset_index(level=[0,1])
df_top10_cities_per_race

In [ ]:
df_top10_cities_per_race = pd.merge(df_top10_cities_per_race,df_cities, on='city')

In [ ]:
df_top10_cities_per_race = df_top10_cities_per_race.rename(columns={"killings_x":"killings", "killings_y":"total_killings"})

In [ ]:
df_top10_cities_per_race

In [ ]:
df_top10_cities_per_race['pct_per_race'] = df_top10_cities_per_race['killings'] * 100 / df_top10_cities_per_race['total_killings']

In [ ]:
df_top10_cities_per_race

In [ ]:
fig = px.bar(df_top10_cities_per_race, x = 'city', y = 'pct_per_race', color = 
    'race', barmode = 'stack')
fig.update_layout(title = "Education Level Customers' Composition",
     xaxis_title = 'Customer Condition', yaxis_title = 'Counts', 
     width = 800, height = 600)
fig.show()

In [ ]:
df_top10_cities_per_race.groupby('city').agg(totalpercent=('pct_per_race', 'sum'))

In [ ]:
df_fatalities

# Create a Choropleth Map of Police Killings by US State

Which states are the most dangerous? Compare your map with your previous chart. Are these the same states with high degrees of poverty? 

In [ ]:
df_killings_per_state = df_fatalities.groupby('state').agg(killings=('year', 'count'))


In [ ]:
usstates =  pd.read_csv('usstates.csv')

In [ ]:
usstates


In [ ]:
df_killings_per_state = pd.merge(df_killings_per_state, usstates, on='state')

In [ ]:
df_killings_per_state

In [ ]:
import plotly.express as px

states = df_killings_per_state.stataname.to_list
print (type(states))
fig = px.choropleth(locations=states[:3], locationmode="USA-states", color=[1,2,3], scope="usa")
fig.show()

In [ ]:
dftest = pd.read_csv('statetest.csv')
dftest

In [ ]:
dftest.c


In [ ]:
import plotly.express as px
# fig = px.choropleth(locations=["CA", "TX", "NY"], locationmode="USA-states", color=[21,122,23], scope="usa")
fig = px.choropleth(dftest.state, locationmode="USA-states", color=dftest.cunt, scope="usa")
fig.show()

In [ ]:
world_map = px.choropleth(df_killings_per_state,
                          locations=df_killings_per_state.stataname,
                          color='killings', 
                          hover_name='killings', 
                          locationmode = 'USA-states',
                          color_continuous_scale=px.colors.sequential.matter)

world_map.update_layout(coloraxis_showscale=True,)

world_map.show()

In [ ]:
df = pd.read_csv('us_exports.csv')

for col in df.columns:
    df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

df['text'] = df['state'] + '<br>' +\
    'Beef '+df['beef']+' Dairy '+df['dairy']+'<br>'+\
    'Fruits '+df['total fruits']+' Veggies ' + df['total veggies']+'<br>'+\
    'Wheat '+df['wheat']+' Corn '+df['corn']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['code'],
        z = df['total exports'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "Millions USD"
        )
    ) ]

layout = dict(
        title = '2011 US Agriculture Exports by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)',
        ),
    )
    
fig = dict( data=data, layout=layout )

url = py.plot( fig, filename='d3-cloropleth-map' )

In [ ]:
import plotly.figure_factory as ff

import numpy as np
import pandas as pd

df_sample = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/laucnty16.csv')
df_sample['State FIPS Code'] = df_sample['State FIPS Code'].apply(lambda x: str(x).zfill(2))
df_sample['County FIPS Code'] = df_sample['County FIPS Code'].apply(lambda x: str(x).zfill(3))
df_sample['FIPS'] = df_sample['State FIPS Code'] + df_sample['County FIPS Code']

colorscale = ["#f7fbff","#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
              "#85bcdb","#6baed6","#57a0ce","#4292c6","#3082be","#2171b5","#1361a9",
              "#08519c","#0b4083","#08306b"]
endpts = list(np.linspace(1, 12, len(colorscale) - 1))
fips = df_sample['FIPS'].tolist()
values = df_sample['Unemployment Rate (%)'].tolist()

fig = ff.create_choropleth(
    fips=fips, values=values,
    binning_endpoints=endpts,
    colorscale=colorscale,
    show_state_data=False,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9, title='USA by Unemployment %',
    legend_title='% unemployed'
)

fig.layout.template = None
fig.show()

# Number of Police Killings Over Time

Analyse the Number of Police Killings over Time. Is there a trend in the data? 

In [ ]:
df_fatalities['year'] = pd.DatetimeIndex(df_fatalities['date']).year
df_fatalities['month'] = pd.DatetimeIndex(df_fatalities['date']).month 

In [ ]:
df_fatalities

In [ ]:
df_killings_per_year = df_fatalities.groupby('year').agg(killings=('date', 'count'))
df_killings_per_year

Dataset is too small, only 2 full years, 2017 has only 7 months, but based on the average monthly killings in 2017, the police seems well on its way to reach its quota.


# Epilogue

Now that you have analysed the data yourself, read [The Washington Post's analysis here](https://www.washingtonpost.com/graphics/investigations/police-shootings-database/).